# Tareas
1) Desde un notebook, script para llamar a ecobici y guardar alguna info SIMPLE (ej: cuántos slots ocupados) en csv.
2) Agregar el secret para usar en github actions
3) Configurar la Github Action para que haga un commit (en una branch en particular? O en main?)
4) Luego de probar que funciona, agregar el cron a la github action
5) Mergear a main el nuevo código

In [32]:
import pandas as pd
from datetime import datetime, UTC
import requests

In [9]:
class TransportBAClient():
    def __init__(self):
        self.get_stations_url = "https://apitransporte.buenosaires.gob.ar/ecobici/gbfs/stationInformation?"
        self.get_stations_status_url = "https://apitransporte.buenosaires.gob.ar/ecobici/gbfs/stationStatus?"
        self.client = requests.Session()
        self.query_params = {
            "client_id": "868ff25380304c3db374a9b6a02798b7",
            "client_secret": "B2da9e8277b847Fea74ff5ad30022464"
        }
        
    def GetStations(self):
        return self.__get(self.get_stations_url)
    
    def GetStationStatus(self):
        return self.__get(self.get_stations_status_url)
    
    def __get(self, url):
        response = self.client.get(url, params=self.query_params)
        if response.status_code >= 300:
            raise requests.exceptions.HTTPError
        data = response.json()
        return data["data"]["stations"]


In [20]:
def preprocessStations(stations_df):
    stations_df = stations_df[["station_id", "groups"]]
    return stations_df.explode("groups")

def preprocessStationsStatus(stations_status_df):
    return station_status_df[
        ["station_id",
         "num_bikes_available", 
         "num_bikes_disabled", 
         "num_docks_available", 
         "num_docks_disabled"]
    ]

In [13]:
transportBAClient = TransportBAClient()
stations = transportBAClient.GetStations()
station_status = transportBAClient.GetStationStatus()

stations_df = pd.DataFrame.from_dict(stations)
station_status_df = pd.DataFrame.from_dict(station_status)

stations_df = preprocessStations(stations_df)
station_status_df = preprocessStationsStatus(station_status_df)
join = pd.merge(stations_df, station_status_df, on="station_id")

"""
# Load or create CSV
df = pd.DataFrame(columns=["job_run_at", "value"])

# Append new row
df.loc[len(df)] = [datetime.utcnow().isoformat(), value]
df.to_csv("data.csv", index=False)"""

'\n# Load or create CSV\ndf = pd.DataFrame(columns=["timestamp", "value"])\n\n# Append new row\ndf.loc[len(df)] = [datetime.utcnow().isoformat(), value]\ndf.to_csv("data.csv", index=False)'

In [44]:
class CSVSaver():
    def __init__(self):
        self.schema = [
            "job_run_at",
            "station_id",
            "groups",
            "num_bikes_available", 
            "num_bikes_disabled", 
            "num_docks_available", 
            "num_docks_disabled",
        ]
    
    def AppendWithJobRunAt(self, new_rows, filename):
        try:
            df = pd.read_csv(filename)
        except FileNotFoundError:
            df = pd.DataFrame(columns=self.schema)
        new_rows["job_run_at"] = datetime.now(UTC).isoformat()
        new_rows["job_run_at"] = pd.to_datetime(new_rows["job_run_at"])
        df = pd.concat([df, new_rows], ignore_index=True)
        df.to_csv(filename, index=False)

In [45]:
csvSaver = CSVSaver()
csvSaver.AppendWithJobRunAt(join, "../data/stations.csv")